# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [163]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [164]:
city_path="../WeatherPy/output_data/cities.csv"
city_df = pd.read_csv(city_path)

In [165]:
labels = ["Unnamed: 0", "index"]
city_df.drop(columns = labels, inplace=True)

In [166]:
city_df.head()

,City,Longitude,Latitude,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date,CityID
0,Clyde River,-68.5914,70.4692,-3.77,84.0,100.0,16.11,CA,1.638131e+09,5924351.0
1,Jamestown,-79.2353,42.0970,36.72,100.0,90.0,9.22,US,1.638131e+09,5122534.0
2,Nainital,79.4469,29.3974,39.52,65.0,0.0,4.14,IN,1.638131e+09,1262117.0
3,Hasaki,140.8333,35.7333,34.20,48.0,2.0,0.89,JP,1.638131e+09,2112802.0
4,Chongwe,28.6820,-15.3292,66.88,49.0,3.0,4.29,ZM,1.638131e+09,917688.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [219]:
# Customize the size of the figure
figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
}
center = (36.863140, -30.015778)

In [220]:
# Create GMAP with heatmap layer

locations = city_df[['Latitude', 'Longitude']]
humidity = city_df['Humidity']

#Create heatmap layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,
                                    max_intensity=100, point_radius=2)

In [221]:
#Plot base layer of map
fig = gmaps.figure(layout=figure_layout, center=center, zoom_level=2.3)

#Add heatmap and markers layer
fig.add_layer(heatmap_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [222]:
city_df.keys()

Index(['City', 'Longitude', 'Latitude', 'MaxTemp', 'Humidity', 'Cloudiness',
       'WindSpeed', 'Country', 'Date', 'CityID'],
      dtype='object')

In [223]:
# # Filter for cities with 70 > MaxTemp < 80, Humidity < 50. Windspeed < 8 , Cloudiness < 50
vaca_df = city_df[(city_df['MaxTemp'] < 80) & (city_df['MaxTemp'] > 70) & (city_df['Humidity'] < 50)
                  & (city_df["WindSpeed"] < 8) & (city_df["Cloudiness"] < 50)]

In [224]:
vaca_df

,City,Longitude,Latitude,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date,CityID
45,Saint George,-113.5841,37.1041,75.67,23.0,1.0,0.00,US,1.638131e+09,5546220.0
78,Raja,25.6780,8.4596,79.77,31.0,7.0,3.98,SS,1.638131e+09,368246.0
116,Manyana,21.7167,-23.4000,75.67,42.0,1.0,6.64,BW,1.638132e+09,933416.0
217,Lompoc,-120.4579,34.6391,79.11,18.0,1.0,5.75,US,1.638131e+09,5367788.0
303,Pacific Grove,-121.9166,36.6177,73.51,35.0,1.0,5.75,US,1.638132e+09,5380437.0
333,Agadez,10.2500,19.7500,75.07,18.0,5.0,5.93,NE,1.638132e+09,2448083.0
358,Mongo,18.6930,12.1844,78.80,15.0,22.0,3.91,TD,1.638132e+09,2427637.0
421,Sonoita,-112.8333,31.8500,79.30,17.0,0.0,5.14,MX,1.638132e+09,3982837.0
508,San Jose,-121.8950,37.3394,71.42,46.0,20.0,3.44,US,1.638132e+09,5392171.0
558,Pahrump,-115.9839,36.2083,71.65,11.0,0.0,1.95,US,1.638132e+09,5509851.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [225]:
hotel_df = pd.DataFrame()

In [226]:
hotel_df['Hotel Name'] = ""
hotel_df['City'] = ""
hotel_df['Rating'] = "" 
hotel_df["Lat"] = ""
hotel_df["Lng"] = ""
# hotel_df["LatLong"] = list(zip(vaca_df.Latitude, vaca_df.Longitude))

In [227]:
hotel_df

,Hotel Name,City,Rating,Lat,Lng


In [228]:
# # Remove parantheses from tuple and print
# f"{('lat','long')[0]},{('lat','long')[1]}"

# # Remove parantheses from tuple and print using lambda across tuple list
# hotel_df["LatLong"].apply(lambda x: x[0])

In [241]:
vaca_df

,City,Longitude,Latitude,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date,CityID
45,Saint George,-113.5841,37.1041,75.67,23.0,1.0,0.00,US,1.638131e+09,5546220.0
78,Raja,25.6780,8.4596,79.77,31.0,7.0,3.98,SS,1.638131e+09,368246.0
116,Manyana,21.7167,-23.4000,75.67,42.0,1.0,6.64,BW,1.638132e+09,933416.0
217,Lompoc,-120.4579,34.6391,79.11,18.0,1.0,5.75,US,1.638131e+09,5367788.0
303,Pacific Grove,-121.9166,36.6177,73.51,35.0,1.0,5.75,US,1.638132e+09,5380437.0
333,Agadez,10.2500,19.7500,75.07,18.0,5.0,5.93,NE,1.638132e+09,2448083.0
358,Mongo,18.6930,12.1844,78.80,15.0,22.0,3.91,TD,1.638132e+09,2427637.0
421,Sonoita,-112.8333,31.8500,79.30,17.0,0.0,5.14,MX,1.638132e+09,3982837.0
508,San Jose,-121.8950,37.3394,71.42,46.0,20.0,3.44,US,1.638132e+09,5392171.0
558,Pahrump,-115.9839,36.2083,71.65,11.0,0.0,1.95,US,1.638132e+09,5509851.0


In [246]:
maps_base_url ="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for k,v in vaca_df.iterrows():
    params = {
    "location" : f"{v.Latitude},{v.Longitude}",
    "radius" : "5000",
    "type" : "lodging",
    "key" :gkey
    }
    
    print(f"Retrieving Results for {v.City}, {v.Country}.")
    
    # assemble url and make API request
    response = requests.get(maps_base_url, params=params).json()
    
    # extract results
    results = response['results']
         
    try:
        print(f"Closest hotel is {results[0]['name']}.")

        hotel_df.loc[k, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[k, 'City'] = results[0]["plus_code"]["compound_code"]
        hotel_df.loc[k, 'Rating'] = results[0]['rating']
        hotel_df.loc[k, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[k, 'Lng'] = results[0]['geometry']['location']['lng']

    except (KeyError, IndexError):
        print(f"No hotel results within 5,000 meters of {v.City}, {v.Country}.")

    print("------------------------------------")


Retrieving Results for Saint George, US.
Closest hotel is Best Western Plus Abbey Inn.
------------------------------------
Retrieving Results for Raja, SS.
No hotel results within 5,000 meters of Raja, SS.
------------------------------------
Retrieving Results for Manyana, BW.
No hotel results within 5,000 meters of Manyana, BW.
------------------------------------
Retrieving Results for Lompoc, US.
Closest hotel is Holiday Inn Express Lompoc, an IHG Hotel.
------------------------------------
Retrieving Results for Pacific Grove, US.
Closest hotel is Lovers Point Inn.
------------------------------------
Retrieving Results for Agadez, NE.
No hotel results within 5,000 meters of Agadez, NE.
------------------------------------
Retrieving Results for Mongo, TD.
Closest hotel is Résidences du Programme FIDA.
------------------------------------
Retrieving Results for Sonoita, MX.
Closest hotel is Hotel Nora.
------------------------------------
Retrieving Results for San Jose, US.
Clos

In [231]:
hotel_df

,Hotel Name,City,Rating,Lat,Lng
45,Best Western Plus Abbey Inn,"3CQ8+J7 St. George, UT, USA",4.6,37.089025,-113.584267
217,"Holiday Inn Express Lompoc, an IHG Hotel","MG7R+2J Lompoc, CA, USA",4.2,34.662502,-120.458466
303,Lovers Point Inn,"J3GM+73 Pacific Grove, CA, USA",3.7,36.625682,-121.917305
358,Résidences du Programme FIDA,"5MMV+F7 Mongo, Chad",3.5,12.183684,18.693246
421,Hotel Nora,"V45W+7X Sonoyta, Sonora, Mexico",4,31.858219,-112.852521
508,The Westin San Jose,"84J6+8Q San Jose, CA, USA",4.2,37.330844,-121.88804
558,Best Western Pahrump Oasis,"623C+HV Pahrump, NV, USA",4.1,36.203983,-115.977797


In [232]:
# # split hotel_city['City]' to create city and country columns and
# #             remove unnecessary characters

test_df = pd.DataFrame()

test_df["City"] = hotel_df["City"].apply(lambda x: x[8:])
test_df["City"] = test_df["City"].apply(lambda x: x[:-5])
test_df["City"][358] = test_df["City"][358][:-1]
test_df["City"][421] = test_df["City"][421][:-3]

test_df["Country"] = hotel_df["City"].apply(lambda x: x[8:])
test_df["Country"] = test_df["Country"].apply(lambda x: x[-6:])
test_df["Country"][45] = test_df["Country"][45][3:]
test_df["Country"][217] = test_df["Country"][217][3:]
test_df["Country"][303] = test_df["Country"][303][3:]
test_df["Country"][508] = test_df["Country"][508][3:]
test_df["Country"][558] = test_df["Country"][558][3:]
test_df["Country"][358] = test_df["Country"][358][2:]

test_df

,City,Country
45,"St. George, UT",USA
217,"Lompoc, CA",USA
303,"Pacific Grove, CA",USA
358,Mongo,Chad
421,"Sonoyta, Sonora",Mexico
508,"San Jose, CA",USA
558,"Pahrump, NV",USA


In [233]:
hotel_df['City']=test_df['City']
hotel_df['Country']=test_df['Country']

In [234]:
hotel_df

,Hotel Name,City,Rating,Lat,Lng,Country
45,Best Western Plus Abbey Inn,"St. George, UT",4.6,37.089025,-113.584267,USA
217,"Holiday Inn Express Lompoc, an IHG Hotel","Lompoc, CA",4.2,34.662502,-120.458466,USA
303,Lovers Point Inn,"Pacific Grove, CA",3.7,36.625682,-121.917305,USA
358,Résidences du Programme FIDA,Mongo,3.5,12.183684,18.693246,Chad
421,Hotel Nora,"Sonoyta, Sonora",4,31.858219,-112.852521,Mexico
508,The Westin San Jose,"San Jose, CA",4.2,37.330844,-121.88804,USA
558,Best Western Pahrump Oasis,"Pahrump, NV",4.1,36.203983,-115.977797,USA


In [239]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [240]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

fig = gmaps.figure(layout=figure_layout, center=center, zoom_level=2.4)
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))